In [46]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

import ml_utils

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [67]:
name = "Shaun McGregor"
show_eda = False

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
226,0,0.550,0.143,0.197,0.468,0.545,0.085,0.840,0.382,0.524,...,0.177,0.498,0.296,0.058,0.998,0.438,0.522,0.492,0.601,0.160
188,0,0.427,0.637,0.339,0.177,0.016,0.876,0.855,0.174,0.638,...,0.496,0.973,0.581,0.077,0.311,0.382,0.038,0.899,0.518,0.996
184,0,0.850,0.982,0.457,0.361,0.853,0.199,0.385,0.092,0.367,...,0.194,0.446,0.456,0.568,0.494,0.871,0.460,0.968,0.307,0.633
82,0,0.654,0.980,0.241,0.674,0.920,0.983,0.076,0.886,0.380,...,0.747,0.250,0.659,0.115,0.379,0.630,0.939,0.267,0.205,0.557
35,1,0.842,0.339,0.893,0.036,0.378,0.477,0.485,0.485,0.602,...,0.457,0.930,0.684,0.340,0.308,0.536,0.638,0.414,0.530,0.785


In [4]:
if show_eda:
    eda = ml_utils.edaDF(df,'target')
    eda.fullEDA()

Output()

In [76]:
# These values have already been scaled
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))

scaler = StandardScaler()
forest = RandomForestClassifier(n_jobs=-1)
tree_para = {
    'forest__min_samples_split':[2,3,4,5,6,7,8,9],
    # 'forest__max_depth':[1,2,3,4,5,6,7,8,9,10],
    'forest__min_impurity_decrease':[.01,.011,.012,.013,.014],
    'forest__ccp_alpha': [.01,.02,.03,.04]
    }
pipe = Pipeline([("forest", forest)])

gridSearch = GridSearchCV(estimator=pipe, param_grid=tree_para, cv=10, n_jobs=-1)


gridSearch.fit(X, y.ravel())
best = gridSearch.best_estimator_

### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [77]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


FileNotFoundError: [Errno 2] No such file or directory: 'testing.csv'

### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li>Tested the ccp_alpha with values from .01 - .5, small values increased continued to improve the overfitting without negatively affecting the accuracy
<li>Tested several max depths, 4-5 seemed like the correct range 
</ul>